In [1]:
#importing the necessary modules
import collections
import gym
import numpy as np
import tensorflow as tf
import tqdm
import scipy.signal
import os
from PIL import Image
from PIL import ImageDraw 
from PIL import ImageFont

from helper import *

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 2.3.1


In [2]:
from datetime import datetime

# Directory that will store all the figures and training checkpoints
dir_name = "keras_train_" + datetime.now().strftime("%m%d-%H%M%S")

In [3]:
# initialize the directories' names to save the models for this particular seed
model_path = dir_name+'/model'
frame_path = dir_name+'/frames'

# create the directories
if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(frame_path):
    os.makedirs(frame_path)

In [2]:
#Defining the Bandit environment

class dependent_bandit():
    # the bandit takes as input the difficulty level for the task or if the choice of reward probability is uniform
    def __init__(self,difficulty):
        self.num_actions = 2
        self.difficulty = difficulty
        self.reset()

    # Choosing the reward probabilities for the left and right choice    
    def reset(self):
        self.timestep = 0
        if self.difficulty == 'easy': bandit_prob = np.random.choice([0.9,0.1])
        if self.difficulty == 'medium': bandit_prob = np.random.choice([0.75,0.25])
        if self.difficulty == 'hard': bandit_prob = np.random.choice([0.6,0.4])
        if self.difficulty == 'uniform': bandit_prob = np.random.uniform()
        
        self.bandit = np.array([bandit_prob,1 - bandit_prob])
  
    def step(self,action):
        
        self.timestep += 1

        # Finding the reward probability of the chosen action
        bandit = self.bandit[action]

        #choosing a random number between 0 and 1
        result = np.random.uniform()

        # Reward is provided according to the reward probability of the chosen action
        if result < bandit:
            #return a positive reward.
            reward = 1
        else:
            #return a negative reward.
            reward = 0

        # End the episode after 100 trials
        if self.timestep > 99: 
            done = True
        else: done = False
            
        return reward,done,self.timestep

In [3]:
class A2C_LSTM(tf.keras.Model):
    """Combined actor-critic network."""

    def __init__(
        self, 
        num_actions: int, 
        num_hidden_units: int):
        """Initialize."""
        super().__init__()

        self.common_LSTM = layers.LSTM(num_hidden_units, return_state=True, return_sequences=True)
        self.actor = layers.Dense(num_actions)
        self.critic = layers.Dense(1)

    def call(self, inputs: tf.Tensor, lstm_state: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor]:

        x, state_h, state_c = self.common_LSTM(inputs, initial_state=lstm_state)
        return self.actor(x), self.critic(x), state_h, state_c

In [4]:
env = dependent_bandit('uniform')
num_actions = 2 
num_hidden_units = 48

model = A2C_LSTM(num_actions, num_hidden_units)

In [5]:
def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Returns state, reward and done flag given an action."""

    reward, done, timestep = env.step(action)
    return (np.array(reward, np.float32), 
        np.array(done, np.bool), 
        np.array(timestep, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
    return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.bool, tf.int32])

In [37]:
def run_episode(
    initial_obs: tf.Tensor,
    initial_state: tf.Tensor,
    model: tf.keras.Model, 
    max_steps: int) -> List[tf.Tensor]:
    """Runs a single episode to collect training data."""
    
    obs_seq = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    actions = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    rew_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    timesteps = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    rewards = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    
    obs = initial_obs
    state = initial_state
        
    for t in tf.range(max_steps):
        
        obs_seq = obs_seq.write(t, obs)
        
        # Convert state into a batched tensor (batch size = 1)
        obs = tf.expand_dims(tf.expand_dims(obs, 0), 0)

        # Run the model and to get action probabilities and critic value
        action_logits_t, value, temp_h, temp_c = model(obs, state)
        
        print(value)
        
        values = values.write(t, tf.squeeze(value))
        
        # LSTM state for the next timestep
        state = [temp_h, temp_c]
        
        # Sample next action from the action probability distribution
        action = tf.random.categorical(tf.expand_dims(tf.squeeze(action_logits_t),0), 1)[0, 0]

        # Apply action to the environment to get next state and reward
        reward, done, timestep = tf_env_step(action)

        # Observation for the next step
        obs = tf.concat([[reward], tf.one_hot(action, num_actions, dtype=tf.float32)],0)

        # Store reward
        rewards = rewards.write(t, reward)

        # Store action
        actions = actions.write(t, tf.cast(action, tf.int32)) 

        # Store timestep
        timesteps = timesteps.write(t, timestep)

        # Store reward probability
        rew_probs = rew_probs.write(t, env.bandit)
        
        if done:
            break
    
    obs_seq = obs_seq.stack()
    actions = actions.stack()
    timesteps = timesteps.stack()
    rew_probs = rew_probs.stack()
    rewards = rewards.stack()
    values = values.stack()
  
    return obs_seq, actions, timesteps, rew_probs, rewards, values

In [7]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float = 0.9, 
    standardize: bool = False) -> tf.Tensor:
    """Compute expected returns per timestep."""

    n = tf.shape(rewards)[0]
    returns = tf.TensorArray(dtype=tf.float32, size=n)

    # Start from the end of `rewards` and accumulate reward sums
    # into the `returns` array
    rewards = tf.cast(rewards[::-1], dtype=tf.float32)
    discounted_sum = tf.constant(0.0)
    discounted_sum_shape = discounted_sum.shape
    for i in tf.range(n):
        reward = rewards[i]
        discounted_sum = reward + gamma * discounted_sum
        discounted_sum.set_shape(discounted_sum_shape)
        returns = returns.write(i, discounted_sum)
    returns = returns.stack()[::-1]

    if standardize:
        returns = ((returns - tf.math.reduce_mean(returns)) / 
                   (tf.math.reduce_std(returns) + eps))

    return returns

In [8]:
def compute_loss_policy(
    action_probs: tf.Tensor,
    probs: tf.Tensor,  
    advantages: tf.Tensor,
    entropy_coef: float = 0.05) -> tf.Tensor:
    """Computes the combined actor-critic loss."""
    
    policy_loss = - tf.math.reduce_sum(tf.math.log(action_probs) * advantages)
    
    entropy_loss = - tf.math.reduce_sum(probs * tf.math.log(probs))

    return policy_loss - entropy_coef*entropy_loss

In [9]:
def compute_loss_value(
    values: tf.Tensor,  
    returns: tf.Tensor,
    value_coef: float = 0.05) -> tf.Tensor:
    """Computes the combined actor-critic loss."""

    val_err = returns - values
    
    value_loss = 0.5 * tf.math.reduce_sum(tf.math.square(val_err))

    return value_coef*value_loss

In [10]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.014)

@tf.function
def train_step(
    obs: tf.Tensor,
    actions: tf.Tensor,
    returns: tf.Tensor,
    advantages: tf.Tensor,
    initial_state: tf.Tensor,
    model: tf.keras.Model) -> tf.Tensor:
    """Runs a model training step."""
    
    state = initial_state

    obs = tf.expand_dims(obs, 0)
    
    with tf.GradientTape() as tape:

        logits, values, _, _ = model(obs, state) 
        
        logits = tf.squeeze(logits)
        values = tf.squeeze(values)
        
        probs = tf.nn.softmax(logits)
        action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        
        for t in tf.range(len(actions)):
            action_probs = action_probs.write(t, probs[t, actions[t]])
            
        action_probs = action_probs.stack()

        # Convert training data to appropriate TF tensor shapes
        action_probs, values, returns, advantages = [
            tf.expand_dims(x, 1) for x in [action_probs, values, returns, advantages]] 
        
        # Calculating loss values to update our network
        loss = compute_loss_policy(action_probs, probs, advantages) + compute_loss_value(values, returns)
    
    # Compute the gradients from the loss
    grads = tape.gradient(loss, model.trainable_variables)
    grads, grad_norms = tf.clip_by_global_norm(grads,999.0)

    # Apply the gradients to the model's parameters
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [11]:
def create_gif(episode_frames):
    img_list = []
    ep_reward = [0,0]
    for i in range(len(episode_frames[0])):
        img_list.append(set_image_bandit(ep_reward,episode_frames[1][i],
                                         episode_frames[2][i],episode_frames[3][i]))
        ep_reward[episode_frames[2][i]] += episode_frames[0][i]
        
    return img_list

In [14]:
%%time

max_episodes = 20001
max_steps_per_episode = 100
train = True
running_reward = 0
# Discount factor for future rewards
gamma = 0.9

with tqdm.trange(max_episodes) as t:
    for i in t:
        env.reset()
        rand_act = np.random.choice([0,1])
        init_rew = 1 if np.random.uniform() < env.bandit[rand_act] else 0
        initial_obs = tf.concat([[init_rew], 
                                 tf.one_hot(rand_act, num_actions, dtype=tf.float32)],0)
        initial_state = [tf.zeros([1,num_hidden_units],tf.float32),
                        tf.zeros([1,num_hidden_units],tf.float32)]
        # Run the model for one episode to collect training data
        if train==True:
            obs_seq, actions, timesteps, rew_probs, rewards, values = run_episode(
                initial_obs, initial_state, model, max_steps_per_episode)
            
            returns = get_expected_return(rewards, gamma)
            
            advantages = returns - values
            
            episode_loss = train_step(obs_seq, actions, returns, advantages, initial_state, model)

        else: 
            obs_seq, actions, timesteps, rew_probs, rewards, values = run_episode(
                initial_obs, initial_state, model, max_steps_per_episode)

        episode_reward = tf.math.reduce_sum(rewards)

        t.set_description(f'Episode {i}')
        t.set_postfix(
            episode_reward=episode_reward, episode_loss=episode_loss)

        # Show average episode reward every 10 episodes
        if i % 100 == 0:
            episode_frames = [rewards.numpy(),rew_probs.numpy(),
                              actions.numpy(),timesteps.numpy()]
            img_list = create_gif(episode_frames)
            ep_type = "/train_" if train == True else "/test_"
            make_gif(img_list,frame_path+ep_type+str(i)+'.gif',
                            duration=len(img_list)*0.1,true_image=True)
          
            if i % 1000 == 0 and train == True:
                model.save_weights(model_path+'/model-'+str(i)+'/model-'+str(i))
                
print(f'\n episode {i}: average reward: {running_reward:.2f}!')

t:  10%|█         | 10/100 [00:00<00:00, 95.99it/s, now=None]

MoviePy - Building file keras_train_1006-182031/frames/train_0.gif with imageio.



t:  13%|█▎        | 13/100 [00:00<00:00, 119.49it/s, now=None]

MoviePy - Building file keras_train_1006-182031/frames/train_100.gif with imageio.



t:   9%|▉         | 9/100 [00:00<00:01, 88.99it/s, now=None]

...

MoviePy - Building file keras_train_1006-182031/frames/train_19900.gif with imageio.



t:  14%|█▍        | 14/100 [00:00<00:00, 129.70it/s, now=None]

MoviePy - Building file keras_train_1006-182031/frames/train_20000.gif with imageio.



Episode 20000: 100%|██████████| 20001/20001 [3:15:03<00:00,  1.71it/s, episode_loss=tf.Tensor(-6.3868074, shape=(), dtype=float32), episode_reward=tf.Tensor(54.0, shape=(), dtype=float32)]


 episode 20000: average reward: 0.00!
CPU times: user 3h 27min 19s, sys: 3min 33s, total: 3h 30min 53s
Wall time: 3h 15min 3s
